In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
hyperparameters = {'train_steps': 10, 'model_name': 'DeepFM'}

In [ ]:
import sagemaker

inputs = sagemaker.Session().upload_data(path='/home/ec2-user/SageMaker/deepctr_sagemaker/data', key_prefix='DEMO-pytorch-deepctr')
print(inputs)

In [ ]:
from sagemaker.pytorch import PyTorch

instance_type = 'local'
# instance_type = 'ml.p3.2xlarge'  # Executes training in a ml.p2.xlarge/ml.p3.2xlarge/ml.p3.8xlarge instance

estimator = PyTorch(entry_point='train_pytorch.py',
                       source_dir='.',
                       instance_type=instance_type, 
                       instance_count=2,  # 1 or 2 or ...
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='1.12.0',
                       py_version='py38',
                       script_mode=True,
                       model_dir='/opt/ml/model')

estimator.fit({'training': inputs})

## Git Support

In [ ]:
# git_config = {'repo': 'https://github.com/whn09/deepctr_sagemaker.git', 'branch': 'main'}

# estimator = TensorFlow(entry_point='train_pytorch.py',
#                        source_dir='.',
#                        git_config=git_config,
#                        instance_type='ml.p3.2xlarge', # Executes training in a ml.p2.xlarge instance
#                        instance_count=1,
#                        hyperparameters=hyperparameters,
#                        role=role,
#                        framework_version='1.12.0',
#                        py_version='py38',
#                        script_mode=True,
#                        model_dir='/opt/ml/model')

# estimator.fit({'training': inputs})

## Deploy the trained model to an endpoint

The deploy() method creates a SageMaker model, which is then deployed to an endpoint to serve prediction requests in real time. We will use the TensorFlow Serving container for the endpoint, because we trained with script mode. This serving container runs an implementation of a web server that is compatible with SageMaker hosting protocol. The Using your own inference code document explains how SageMaker runs inference containers.

In [ ]:
instance_type = 'local'
# instance_type = 'ml.m4.xlarge'
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

## Invoke the endpoint

Let's download the training data and use that as input for inference.

In [ ]:
import json

def test_REST_serving():
    '''
    test rest api 
    '''
    null = None
    fea_dict1 = {"I1":null,"I2":3,"I3":260.0,"I4":null,"I5":17668.0,"I6":null,"I7":null,"I8":33.0,"I9":null,"I10":null,"I11":null,"I12":0.0,"I13":null,"C1":"05db9164","C2":"08d6d899","C3":"9143c832","C4":"f56b7dd5","C5":"25c83c98","C6":"7e0ccccf","C7":"df5c2d18","C8":"0b153874","C9":"a73ee510","C10":"8f48ce11","C11":"a7b606c4","C12":"ae1bb660","C13":"eae197fd","C14":"b28479f6","C15":"bfef54b3","C16":"bad5ee18","C17":"e5ba7672","C18":"87c6f83c","C19":null,"C20":null,"C21":"0429f84b","C22":null,"C23":"3a171ecb","C24":"c0d61a5c","C25":null,"C26":null}
    fea_dict2 = {"I1":null,"I2":-1,"I3":19.0,"I4":35.0,"I5":30251.0,"I6":247.0,"I7":1.0,"I8":35.0,"I9":160.0,"I10":null,"I11":1.0,"I12":null,"I13":35.0,"C1":"68fd1e64","C2":"04e09220","C3":"95e13fd4","C4":"a1e6a194","C5":"25c83c98","C6":"fe6b92e5","C7":"f819e175","C8":"062b5529","C9":"a73ee510","C10":"ab9456b4","C11":"6153cf57","C12":"8882c6cd","C13":"769a1844","C14":"b28479f6","C15":"69f825dd","C16":"23056e4f","C17":"d4bb7bd8","C18":"6fc84bfb","C19":null,"C20":null,"C21":"5155d8a3","C22":null,"C23":"be7c41b4","C24":"ded4aac9","C25":null,"C26":null}
    fea_dict3 = {"I1":0.0,"I2":0,"I3":2.0,"I4":12.0,"I5":2013.0,"I6":164.0,"I7":6.0,"I8":35.0,"I9":523.0,"I10":0.0,"I11":3.0,"I12":null,"I13":18.0,"C1":"05db9164","C2":"38a947a1","C3":"3f55fb72","C4":"5de245c7","C5":"30903e74","C6":"7e0ccccf","C7":"b72ec13d","C8":"1f89b562","C9":"a73ee510","C10":"acce978c","C11":"3547565f","C12":"a5b0521a","C13":"12880350","C14":"b28479f6","C15":"c12fc269","C16":"95a8919c","C17":"e5ba7672","C18":"675c9258","C19":null,"C20":null,"C21":"2e01979f","C22":null,"C23":"bcdee96c","C24":"6d5d1302","C25":null,"C26":null}
    fea_dict4 = {"I1":null,"I2":13,"I3":1.0,"I4":4.0,"I5":16836.0,"I6":200.0,"I7":5.0,"I8":4.0,"I9":29.0,"I10":null,"I11":2.0,"I12":null,"I13":4.0,"C1":"05db9164","C2":"8084ee93","C3":"02cf9876","C4":"c18be181","C5":"25c83c98","C6":null,"C7":"e14874c9","C8":"0b153874","C9":"7cc72ec2","C10":"2462946f","C11":"636405ac","C12":"8fe001f4","C13":"31b42deb","C14":"07d13a8f","C15":"422c8577","C16":"36103458","C17":"e5ba7672","C18":"52e44668","C19":null,"C20":null,"C21":"e587c466","C22":null,"C23":"32c7478e","C24":"3b183c5c","C25":null,"C26":null}

    data = {"instances": [fea_dict1,fea_dict2,fea_dict3,fea_dict4]}
    # print(data)

    json_response = predictor.predict(data)
    predictions = json_response  # ['predictions']
#     print(predictions)
    return predictions

In [ ]:
test_REST_serving()

## Delete the endpoint

Let's delete the endpoint we just created to prevent incurring any extra costs.

In [ ]:
predictor.delete_endpoint()